In [0]:

from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import lit

In [0]:

%fs
ls /Volumes/workspace/ecommerce/ecommerce_data

In [0]:
df_oct = spark.read.parquet("/Volumes/workspace/ecommerce/ecommerce_data/parquet/oct/")
df_oct.limit(5).show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5001201392501748>, line 2
      1 df_oct = spark.read.parquet("/Volumes/workspace/ecommerce/ecommerce_data/parquet/oct/")
----> 2 df_oct.limit(5).show()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1123, in DataFrame.show(self, n, truncate, vertical)
   1122 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
-> 1123     print(self._show_string(n, truncate, vertical))

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:876, in DataFrame._show_string(self, n, truncate, vertical)
    859     except ValueError:
    860         raise PySparkTypeError(
    861             errorClass="NOT_BOOL",
    862             messageParameters={
   (...)
    865             },
    866         )
    868 table, _ = DataFra

In [0]:
%sql
WITH daily AS (
  SELECT
    DATE(event_time) AS event_date,
    SUM(price) AS rev
  FROM ecommerce.silver.events
  WHERE event_type = 'purchase'
  GROUP BY DATE(event_time)
)
SELECT
  event_date,
  rev,
  AVG(rev) OVER (
    ORDER BY event_date
    ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
  ) AS ma7
FROM daily
ORDER BY event_date;

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:139)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:139)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:136)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:721)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:441)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:441)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:

%sql
SELECT
  category_code,
  SUM(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END) AS views,
  SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) AS purchases,
  ROUND(
    SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) * 100.0 /
    NULLIF(SUM(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END), 0),
    2
  ) AS conversion_rate
FROM ecommerce.silver.events
GROUP BY category_code
ORDER BY conversion_rate DESC;

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:440)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:486)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:768)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:512)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:632)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$5(UsageLogging.scala:659)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:117)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:348)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:59)
	at com.databricks.logging.AttributionContext$.withValue(Att

In [0]:

%sql
WITH customer_metrics AS (
  SELECT
    user_id,
    COUNT(*) AS cnt,
    SUM(price) AS total_spent
  FROM ecommerce.silver.events
  WHERE event_type = 'purchase'
  GROUP BY user_id
)
SELECT
  CASE
    WHEN cnt >= 10 THEN 'VIP'
    WHEN cnt >= 5 THEN 'Loyal'
    ELSE 'Regular'
  END AS tier,
  COUNT(*) AS customers,
  ROUND(AVG(total_spent), 2) AS avg_ltv
FROM customer_metrics
GROUP BY tier
ORDER BY avg_ltv DESC;


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:440)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:486)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:768)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:512)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:632)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$5(UsageLogging.scala:659)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:117)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:348)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:59)
	at com.databricks.logging.AttributionContext$.withValue(Att